# Task 5: Matrix Factorization & Explainable Hybrid Recommender

In this notebook, we'll implement and compare matrix factorization techniques with collaborative filtering methods. We'll cover:

1. Data preparation
2. Matrix factorization implementation
3. Generating recommendations
4. Quality evaluation of models
5. Factor interpretation

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set random seed for reproducibility
np.random.seed(42)

## 1. Data Preparation

We'll start by generating (or reusing) a user-item matrix from Task 1, where:
- We have 100 users and 20 courses
- Ratings range from 1 to 5
- 0 indicates that the user has not taken the course

Then we'll split the non-zero ratings into train/test sets (80/20).

In [ ]:
# Define constants
NUM_USERS = 100
NUM_COURSES = 20
SPARSITY = 0.85  # 85% of the matrix will be zero (users haven't taken the course)

# Generate the user-item ratings matrix
# First create a matrix of zeros
ratings = np.zeros((NUM_USERS, NUM_COURSES))

# Determine which elements will have ratings (non-zero values)
mask = np.random.random((NUM_USERS, NUM_COURSES)) > SPARSITY

# Generate ratings (1-5) for the non-zero elements with a positive bias
# Most people give 3-5 stars if they complete a course
ratings[mask] = np.random.choice([1, 2, 3, 4, 5], size=np.sum(mask), p=[0.05, 0.1, 0.2, 0.3, 0.35])

print(f"Generated ratings matrix with shape: {ratings.shape}")
print(f"Sparsity (percentage of zeros): {np.mean(ratings == 0) * 100:.2f}%")
print(f"Number of ratings provided: {np.sum(ratings > 0)}")

# Create course names for interpretability
course_names = [
    "Introduction to Programming", "Advanced Python", "Data Structures", 
    "Algorithms", "Database Systems", "Web Development", "Machine Learning",
    "Deep Learning", "Computer Vision", "Natural Language Processing",
    "Cloud Computing", "DevOps", "Cybersecurity", "Mobile App Development",
    "Game Development", "UI/UX Design", "Project Management", 
    "Software Engineering", "Computer Networks", "Operating Systems"
]

# Split observations into train/test (80/20 random nonzero values)
# Get indices of all rated items (non-zero elements)
user_indices, course_indices = np.where(ratings > 0)
num_ratings = len(user_indices)

# Shuffle the indices
indices = np.arange(num_ratings)
np.random.shuffle(indices)

# Split into train and test
split_idx = int(0.8 * num_ratings)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]

# Create train and test matrices
train_matrix = np.zeros_like(ratings)
test_matrix = np.zeros_like(ratings)

# Fill in the train and test matrices
for i in train_indices:
    user_idx, course_idx = user_indices[i], course_indices[i]
    train_matrix[user_idx, course_idx] = ratings[user_idx, course_idx]

for i in test_indices:
    user_idx, course_idx = user_indices[i], course_indices[i]
    test_matrix[user_idx, course_idx] = ratings[user_idx, course_idx]

print(f"\nTraining matrix has {np.sum(train_matrix > 0)} ratings")
print(f"Testing matrix has {np.sum(test_matrix > 0)} ratings")

# Visualize the training and test matrices
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.spy(ratings, markersize=0.5, aspect='auto')
plt.title('Full Ratings Matrix')
plt.xlabel('Courses')
plt.ylabel('Users')

plt.subplot(1, 3, 2)
plt.spy(train_matrix, markersize=0.5, aspect='auto')
plt.title('Training Matrix')
plt.xlabel('Courses')
plt.ylabel('Users')

plt.subplot(1, 3, 3)
plt.spy(test_matrix, markersize=0.5, aspect='auto')
plt.title('Testing Matrix')
plt.xlabel('Courses')
plt.ylabel('Users')

plt.tight_layout()
plt.show()

## 2. Matrix Factorization Implementation

Now we'll implement a matrix factorization algorithm using stochastic gradient descent (SGD):

- We'll decompose our ratings matrix R into two lower-rank matrices P and Q
- R ≈ P × Q^T, where P represents user factors and Q represents course factors
- We'll optimize using SGD with learning rate and regularization parameters

In [ ]:
# Define the matrix factorization function
def matrix_factorization(R, K, learning_rate=0.005, reg_param=0.02, epochs=100, verbose=True):
    """
    Implement matrix factorization using SGD.
    
    Parameters:
    - R: User-item ratings matrix
    - K: Number of latent factors
    - learning_rate: Learning rate for gradient descent
    - reg_param: Regularization parameter
    - epochs: Number of iterations
    - verbose: Whether to print progress
    
    Returns:
    - P: User factors matrix
    - Q: Item factors matrix
    - error_history: Training error history
    """
    # Get dimensions
    num_users, num_items = R.shape
    
    # Initialize factors matrices with small random values
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    # Create mask for non-zero elements (where ratings exist)
    mask = (R > 0).astype(float)
    
    # Track errors
    error_history = []
    
    # Perform SGD
    for epoch in range(epochs):
        # Calculate predicted ratings
        R_pred = np.dot(P, Q.T)
        
        # Calculate error (only for observed ratings)
        error = mask * (R - R_pred)
        
        # Calculate root mean squared error
        rmse = np.sqrt(np.sum(error**2) / np.sum(mask))
        error_history.append(rmse)
        
        if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
            print(f"Epoch {epoch+1}/{epochs} - RMSE: {rmse:.4f}")
        
        # Update P and Q based on error gradient
        for u in range(num_users):
            for i in range(num_items):
                if mask[u, i] > 0:  # Update only for observed ratings
                    # Calculate error for this rating
                    e_ui = R[u, i] - np.dot(P[u, :], Q[i, :].T)
                    
                    # Update user and item factors
                    P[u, :] += learning_rate * (e_ui * Q[i, :] - reg_param * P[u, :])
                    Q[i, :] += learning_rate * (e_ui * P[u, :] - reg_param * Q[i, :])
    
    return P, Q, error_history

# Set parameters for matrix factorization
K = 5  # Number of latent factors
learning_rate = 0.005
reg_param = 0.02
epochs = 50

# Run matrix factorization
P, Q, error_history = matrix_factorization(train_matrix, K, learning_rate, reg_param, epochs)

# Calculate predicted ratings
R_pred = np.dot(P, Q.T)

# Plot training error history
plt.figure(figsize=(10, 5))
plt.plot(error_history)
plt.title('Matrix Factorization Training Error')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate RMSE on the test set
test_user_indices, test_item_indices = np.where(test_matrix > 0)
test_predictions = np.zeros_like(test_matrix)

for i in range(len(test_user_indices)):
    u = test_user_indices[i]
    j = test_item_indices[i]
    test_predictions[u, j] = np.dot(P[u, :], Q[j, :])

test_rmse = np.sqrt(np.mean((test_matrix[test_matrix > 0] - test_predictions[test_matrix > 0]) ** 2))
print(f"Test RMSE: {test_rmse:.4f}")

# Visualize the original vs. predicted ratings for a sample
plt.figure(figsize=(12, 5))

# Select 10 random test samples
num_samples = min(20, len(test_user_indices))
sample_indices = np.random.choice(range(len(test_user_indices)), num_samples, replace=False)

# Get original and predicted ratings for these samples
original_ratings = []
predicted_ratings = []

for idx in sample_indices:
    u = test_user_indices[idx]
    j = test_item_indices[idx]
    original_ratings.append(test_matrix[u, j])
    predicted_ratings.append(test_predictions[u, j])

# Plot comparison
plt.subplot(1, 2, 1)
plt.scatter(original_ratings, predicted_ratings)
plt.plot([1, 5], [1, 5], 'r--')  # Perfect prediction line
plt.xlim(0.5, 5.5)
plt.ylim(0.5, 5.5)
plt.title('Original vs. Predicted Ratings')
plt.xlabel('Original Rating')
plt.ylabel('Predicted Rating')
plt.grid(True, alpha=0.3)

# Plot error distribution
plt.subplot(1, 2, 2)
errors = np.array(original_ratings) - np.array(predicted_ratings)
plt.hist(errors, bins=10, alpha=0.7)
plt.title('Prediction Error Distribution')
plt.xlabel('Error (Original - Predicted)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Generating Recommendations

Now we'll use our matrix factorization model to generate recommendations:

1. Compute the predicted ratings matrix R_hat = P @ Q.T
2. For a chosen user, find the top-5 courses they haven't taken yet
3. Compare recommendation lists from matrix factorization, user-based CF, and item-based CF

In [ ]:
# Define cosine similarity function (for collaborative filtering methods)
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)

# Compute user similarity matrix
def compute_user_similarity(ratings):
    num_users = ratings.shape[0]
    sim_matrix = np.zeros((num_users, num_users))
    
    for i in range(num_users):
        for j in range(i, num_users):  # Start from i to avoid duplicate calculations
            # Skip if it's the same user
            if i == j:
                sim_matrix[i, j] = 1.0  # Self-similarity is 1
            else:
                # Calculate similarity only where both users rated items
                user_i_ratings = ratings[i].copy()
                user_j_ratings = ratings[j].copy()
                
                # Find courses both users have rated
                common_items = (user_i_ratings > 0) & (user_j_ratings > 0)
                
                # If they have no courses in common, similarity is 0
                if np.sum(common_items) == 0:
                    sim_matrix[i, j] = 0.0
                    sim_matrix[j, i] = 0.0
                else:
                    # Calculate cosine similarity on common items
                    sim = cosine_similarity(
                        user_i_ratings[common_items], 
                        user_j_ratings[common_items]
                    )
                    sim_matrix[i, j] = sim
                    sim_matrix[j, i] = sim  # Similarity is symmetric
    
    return sim_matrix

# Compute course similarity matrix
def compute_course_similarity(ratings):
    num_courses = ratings.shape[1]
    sim_matrix = np.zeros((num_courses, num_courses))
    
    for i in range(num_courses):
        for j in range(i, num_courses):  # Start from i to avoid duplicate calculations
            # Skip if it's the same course
            if i == j:
                sim_matrix[i, j] = 1.0  # Self-similarity is 1
            else:
                # Calculate similarity only where both courses were rated by users
                course_i_ratings = ratings[:, i].copy()
                course_j_ratings = ratings[:, j].copy()
                
                # Find users who rated both courses
                common_users = (course_i_ratings > 0) & (course_j_ratings > 0)
                
                # If no user rated both courses, similarity is 0
                if np.sum(common_users) == 0:
                    sim_matrix[i, j] = 0.0
                    sim_matrix[j, i] = 0.0
                else:
                    # Calculate cosine similarity on common users
                    sim = cosine_similarity(
                        course_i_ratings[common_users], 
                        course_j_ratings[common_users]
                    )
                    sim_matrix[i, j] = sim
                    sim_matrix[j, i] = sim  # Similarity is symmetric
    
    return sim_matrix

# Compute the similarity matrices (for CF methods)
user_similarity = compute_user_similarity(train_matrix)
course_similarity = compute_course_similarity(train_matrix)

# Function to get top-k similar users
def get_top_k_similar_users(user_id, similarity_matrix, k=5):
    """Get the top-k most similar users for a given user."""
    # Get similarity scores for this user with all others
    user_similarities = similarity_matrix[user_id]
    
    # Get indices of top-k similar users (excluding the user itself)
    similar_user_indices = np.argsort(user_similarities)[::-1]
    
    # Remove the user itself (which would have similarity 1.0)
    similar_user_indices = similar_user_indices[similar_user_indices != user_id][:k]
    
    return similar_user_indices

# Function to get user-based CF recommendations
def get_user_based_recommendations(user_id, ratings, user_similarity, k=5, top_n=5):
    """Generate top-n recommendations for user_id based on similar users."""
    # Get the user's ratings
    user_ratings = ratings[user_id]
    
    # Get top-k similar users
    similar_users = get_top_k_similar_users(user_id, user_similarity, k)
    
    # Initialize dictionary to store predicted ratings
    predicted_ratings = {}
    
    # Get courses that the user hasn't rated yet
    unrated_courses = np.where(user_ratings == 0)[0]
    
    # For each unrated course
    for course in unrated_courses:
        # Get ratings of similar users for this course
        similar_user_ratings = []
        similar_user_weights = []
        
        for similar_user in similar_users:
            # If the similar user has rated this course
            if ratings[similar_user, course] > 0:
                similar_user_ratings.append(ratings[similar_user, course])
                similar_user_weights.append(user_similarity[user_id, similar_user])
        
        # If at least one similar user has rated this course
        if len(similar_user_ratings) > 0:
            # Calculate weighted average rating
            weighted_sum = np.sum(np.array(similar_user_ratings) * np.array(similar_user_weights))
            weight_sum = np.sum(similar_user_weights)
            
            if weight_sum > 0:  # Avoid division by zero
                predicted_rating = weighted_sum / weight_sum
                predicted_ratings[course] = predicted_rating
    
    # Sort courses by predicted rating (descending)
    sorted_predictions = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)
    
    # Return top_n recommendations
    return sorted_predictions[:top_n]

# Function to get item-based CF recommendations
def get_item_based_recommendations(user_id, ratings, course_similarity, top_n=5):
    """Generate top-n recommendations for user_id based on item similarity."""
    # Get the user's ratings
    user_ratings = ratings[user_id]
    
    # Get courses that the user hasn't rated yet
    unrated_courses = np.where(user_ratings == 0)[0]
    
    # Get courses that the user has rated
    rated_courses = np.where(user_ratings > 0)[0]
    
    # If user hasn't rated any courses, we can't make item-based recommendations
    if len(rated_courses) == 0:
        return []
    
    # Initialize dictionary to store predicted ratings
    predicted_ratings = {}
    
    # For each unrated course
    for unrated_course in unrated_courses:
        weighted_sum = 0.0
        similarity_sum = 0.0
        
        # For each rated course
        for rated_course in rated_courses:
            # Get similarity between the rated course and the unrated one
            similarity = course_similarity[unrated_course, rated_course]
            
            # Add weighted rating to the sum
            weighted_sum += similarity * user_ratings[rated_course]
            similarity_sum += np.abs(similarity)  # Use absolute similarity as weight
        
        # Calculate predicted rating if there's a similarity sum
        if similarity_sum > 0:
            predicted_ratings[unrated_course] = weighted_sum / similarity_sum
    
    # Sort courses by predicted rating (descending)
    sorted_predictions = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)
    
    # Return top_n recommendations
    return sorted_predictions[:top_n]

# Function to get matrix factorization recommendations
def get_mf_recommendations(user_id, P, Q, original_ratings, top_n=5):
    """Generate top-n recommendations for user_id based on matrix factorization."""
    # Get the user's original ratings
    user_ratings = original_ratings[user_id]
    
    # Get courses that the user hasn't rated yet
    unrated_courses = np.where(user_ratings == 0)[0]
    
    # Calculate predicted ratings for all unrated courses
    predicted_ratings = {}
    for course in unrated_courses:
        predicted_rating = np.dot(P[user_id], Q[course])
        predicted_ratings[course] = predicted_rating
    
    # Sort courses by predicted rating (descending)
    sorted_predictions = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)
    
    # Return top_n recommendations
    return sorted_predictions[:top_n]

# Choose a specific user for recommendations
target_user_id = 10  # User ID 10 (11th user)

# Get recommendations from all three methods
mf_recommendations = get_mf_recommendations(target_user_id, P, Q, train_matrix)
user_based_recommendations = get_user_based_recommendations(target_user_id, train_matrix, user_similarity)
item_based_recommendations = get_item_based_recommendations(target_user_id, train_matrix, course_similarity)

# Print recommendations
print(f"User {target_user_id}'s ratings:")
for i, rating in enumerate(train_matrix[target_user_id]):
    if rating > 0:
        print(f"  Course '{course_names[i]}': {rating}")

print("\nTop-5 recommendations from Matrix Factorization:")
for course_id, predicted_rating in mf_recommendations:
    print(f"  Course '{course_names[course_id]}': {predicted_rating:.2f}")

print("\nTop-5 recommendations from User-Based CF:")
for course_id, predicted_rating in user_based_recommendations:
    print(f"  Course '{course_names[course_id]}': {predicted_rating:.2f}")

print("\nTop-5 recommendations from Item-Based CF:")
for course_id, predicted_rating in item_based_recommendations:
    print(f"  Course '{course_names[course_id]}': {predicted_rating:.2f}")

# Compare recommendations across multiple users
def compare_recommendations(users, P, Q, train_matrix, user_similarity, course_similarity):
    """Compare recommendations from all three methods for multiple users."""
    results = []
    
    for user_id in users:
        mf_recs = get_mf_recommendations(user_id, P, Q, train_matrix)
        user_cf_recs = get_user_based_recommendations(user_id, train_matrix, user_similarity)
        item_cf_recs = get_item_based_recommendations(user_id, train_matrix, course_similarity)
        
        # Extract course IDs
        mf_courses = [rec[0] for rec in mf_recs]
        user_cf_courses = [rec[0] for rec in user_cf_recs]
        item_cf_courses = [rec[0] for rec in item_cf_recs]
        
        # Calculate overlap
        mf_user_cf_overlap = len(set(mf_courses) & set(user_cf_courses))
        mf_item_cf_overlap = len(set(mf_courses) & set(item_cf_courses))
        user_cf_item_cf_overlap = len(set(user_cf_courses) & set(item_cf_courses))
        
        results.append({
            'user_id': user_id,
            'mf_courses': mf_courses,
            'user_cf_courses': user_cf_courses,
            'item_cf_courses': item_cf_courses,
            'mf_user_cf_overlap': mf_user_cf_overlap,
            'mf_item_cf_overlap': mf_item_cf_overlap,
            'user_cf_item_cf_overlap': user_cf_item_cf_overlap
        })
    
    return results

# Compare recommendations for 5 random users
random_users = np.random.choice(NUM_USERS, 5, replace=False)
comparison_results = compare_recommendations(random_users, P, Q, train_matrix, user_similarity, course_similarity)

# Print comparison results
print("\nComparison of recommendations for 5 random users:")
for result in comparison_results:
    user_id = result['user_id']
    print(f"\nUser {user_id}:")
    print(f"  MF recommends: {[course_names[c] for c in result['mf_courses'][:3]]}")
    print(f"  User-CF recommends: {[course_names[c] for c in result['user_cf_courses'][:3]]}")
    print(f"  Item-CF recommends: {[course_names[c] for c in result['item_cf_courses'][:3]]}")
    print(f"  Overlap: MF-UserCF: {result['mf_user_cf_overlap']}, MF-ItemCF: {result['mf_item_cf_overlap']}, UserCF-ItemCF: {result['user_cf_item_cf_overlap']}")

## 4. Quality Evaluation of Models

Let's evaluate the quality of our three recommendation approaches:
1. Calculate RMSE on the test set
2. Implement precision@5 and recall@5 (considering ratings ≥4 as relevant)
3. Compare metrics across all three approaches

In [ ]:
# Calculate predicted ratings for all methods
# For Matrix Factorization
test_user_indices, test_item_indices = np.where(test_matrix > 0)
mf_predictions = np.zeros_like(test_matrix)

for i in range(len(test_user_indices)):
    u = test_user_indices[i]
    j = test_item_indices[i]
    mf_predictions[u, j] = np.dot(P[u, :], Q[j, :])

# Calculate RMSE for MF
mf_rmse = np.sqrt(np.mean((test_matrix[test_matrix > 0] - mf_predictions[test_matrix > 0]) ** 2))

# For User-Based CF
user_cf_predictions = np.zeros_like(test_matrix)

for i in range(len(test_user_indices)):
    u = test_user_indices[i]
    j = test_item_indices[i]
    
    # Get top-k similar users
    similar_users = get_top_k_similar_users(u, user_similarity)
    
    # Get ratings of similar users for this course
    similar_user_ratings = []
    similar_user_weights = []
    
    for similar_user in similar_users:
        # If the similar user has rated this course
        if train_matrix[similar_user, j] > 0:
            similar_user_ratings.append(train_matrix[similar_user, j])
            similar_user_weights.append(user_similarity[u, similar_user])
    
    # If at least one similar user has rated this course
    if len(similar_user_ratings) > 0:
        # Calculate weighted average rating
        weighted_sum = np.sum(np.array(similar_user_ratings) * np.array(similar_user_weights))
        weight_sum = np.sum(similar_user_weights)
        
        if weight_sum > 0:  # Avoid division by zero
            user_cf_predictions[u, j] = weighted_sum / weight_sum
        else:
            # Use the average rating for this course as a fallback
            course_ratings = train_matrix[:, j]
            non_zero_ratings = course_ratings[course_ratings > 0]
            if len(non_zero_ratings) > 0:
                user_cf_predictions[u, j] = np.mean(non_zero_ratings)

# Calculate RMSE for User-Based CF (only where predictions were made)
user_cf_mask = (user_cf_predictions > 0) & (test_matrix > 0)
if np.sum(user_cf_mask) > 0:
    user_cf_rmse = np.sqrt(np.mean((test_matrix[user_cf_mask] - user_cf_predictions[user_cf_mask]) ** 2))
else:
    user_cf_rmse = float('nan')

# For Item-Based CF
item_cf_predictions = np.zeros_like(test_matrix)

for i in range(len(test_user_indices)):
    u = test_user_indices[i]
    j = test_item_indices[i]
    
    # Get courses that the user has rated
    rated_courses = np.where(train_matrix[u, :] > 0)[0]
    
    if len(rated_courses) > 0:
        weighted_sum = 0.0
        similarity_sum = 0.0
        
        # For each rated course
        for rated_course in rated_courses:
            # Get similarity between the rated course and the test course
            similarity = course_similarity[j, rated_course]
            
            # Add weighted rating to the sum
            weighted_sum += similarity * train_matrix[u, rated_course]
            similarity_sum += np.abs(similarity)
        
        # Calculate predicted rating if there's a similarity sum
        if similarity_sum > 0:
            item_cf_predictions[u, j] = weighted_sum / similarity_sum
        else:
            # Use the average rating for this course as a fallback
            course_ratings = train_matrix[:, j]
            non_zero_ratings = course_ratings[course_ratings > 0]
            if len(non_zero_ratings) > 0:
                item_cf_predictions[u, j] = np.mean(non_zero_ratings)

# Calculate RMSE for Item-Based CF (only where predictions were made)
item_cf_mask = (item_cf_predictions > 0) & (test_matrix > 0)
if np.sum(item_cf_mask) > 0:
    item_cf_rmse = np.sqrt(np.mean((test_matrix[item_cf_mask] - item_cf_predictions[item_cf_mask]) ** 2))
else:
    item_cf_rmse = float('nan')

print("RMSE Comparison:")
print(f"  Matrix Factorization: {mf_rmse:.4f}")
print(f"  User-Based CF: {user_cf_rmse:.4f}")
print(f"  Item-Based CF: {item_cf_rmse:.4f}")

# Calculate precision and recall at k=5
def calculate_precision_recall_at_k(recommendations, user_id, test_matrix, k=5, relevant_threshold=4):
    """
    Calculate precision and recall at k for a user's recommendations.
    
    Parameters:
    - recommendations: List of tuples (course_id, predicted_rating)
    - user_id: ID of the target user
    - test_matrix: Test data matrix
    - k: Number of recommendations to consider
    - relevant_threshold: Minimum rating to consider an item relevant
    
    Returns:
    - precision: Precision at k
    - recall: Recall at k
    """
    # Get the user's test ratings
    user_test_ratings = test_matrix[user_id]
    
    # Get the courses the user actually rated in the test set that are relevant
    relevant_courses = set(np.where((user_test_ratings >= relevant_threshold) & (user_test_ratings > 0))[0])
    
    # If no relevant courses in the test set, return 0 for precision and recall
    if len(relevant_courses) == 0:
        return 0, 0
    
    # Get the top k recommended courses
    if len(recommendations) < k:
        recommended_courses = set([rec[0] for rec in recommendations])
    else:
        recommended_courses = set([rec[0] for rec in recommendations[:k]])
    
    # Calculate the number of true positives (recommended courses that are relevant)
    true_positives = len(relevant_courses.intersection(recommended_courses))
    
    # Calculate precision and recall
    precision = true_positives / len(recommended_courses) if recommended_courses else 0
    recall = true_positives / len(relevant_courses) if relevant_courses else 0
    
    return precision, recall

# Evaluate all three methods for all users
mf_precision_sum = 0
mf_recall_sum = 0
user_cf_precision_sum = 0
user_cf_recall_sum = 0
item_cf_precision_sum = 0
item_cf_recall_sum = 0
num_evaluated_users = 0

for user_id in range(NUM_USERS):
    # Skip users who don't have any ratings in the test set
    user_test_ratings = test_matrix[user_id]
    if np.sum(user_test_ratings > 0) == 0:
        continue
    
    # Get recommendations using all three methods
    mf_recs = get_mf_recommendations(user_id, P, Q, train_matrix)
    user_cf_recs = get_user_based_recommendations(user_id, train_matrix, user_similarity)
    item_cf_recs = get_item_based_recommendations(user_id, train_matrix, course_similarity)
    
    # Calculate precision and recall for matrix factorization
    if mf_recs:
        mf_precision, mf_recall = calculate_precision_recall_at_k(mf_recs, user_id, test_matrix)
        mf_precision_sum += mf_precision
        mf_recall_sum += mf_recall
    
    # Calculate precision and recall for user-based recommendations
    if user_cf_recs:
        user_cf_precision, user_cf_recall = calculate_precision_recall_at_k(user_cf_recs, user_id, test_matrix)
        user_cf_precision_sum += user_cf_precision
        user_cf_recall_sum += user_cf_recall
    
    # Calculate precision and recall for item-based recommendations
    if item_cf_recs:
        item_cf_precision, item_cf_recall = calculate_precision_recall_at_k(item_cf_recs, user_id, test_matrix)
        item_cf_precision_sum += item_cf_precision
        item_cf_recall_sum += item_cf_recall
    
    # Increment counter for users with test data
    num_evaluated_users += 1

# Calculate average precision and recall
if num_evaluated_users > 0:
    avg_mf_precision = mf_precision_sum / num_evaluated_users
    avg_mf_recall = mf_recall_sum / num_evaluated_users
    avg_user_cf_precision = user_cf_precision_sum / num_evaluated_users
    avg_user_cf_recall = user_cf_recall_sum / num_evaluated_users
    avg_item_cf_precision = item_cf_precision_sum / num_evaluated_users
    avg_item_cf_recall = item_cf_recall_sum / num_evaluated_users
else:
    avg_mf_precision = avg_mf_recall = avg_user_cf_precision = avg_user_cf_recall = avg_item_cf_precision = avg_item_cf_recall = 0

print("\nPrecision@5 and Recall@5 Comparison (relevant: rating >= 4):")
print(f"  Matrix Factorization - Precision: {avg_mf_precision:.4f}, Recall: {avg_mf_recall:.4f}")
print(f"  User-Based CF - Precision: {avg_user_cf_precision:.4f}, Recall: {avg_user_cf_recall:.4f}")
print(f"  Item-Based CF - Precision: {avg_item_cf_precision:.4f}, Recall: {avg_item_cf_recall:.4f}")

# Create a table for comparison
metrics_df = pd.DataFrame({
    'Method': ['Matrix Factorization', 'User-Based CF', 'Item-Based CF'],
    'RMSE': [mf_rmse, user_cf_rmse, item_cf_rmse],
    'Precision@5': [avg_mf_precision, avg_user_cf_precision, avg_item_cf_precision],
    'Recall@5': [avg_mf_recall, avg_user_cf_recall, avg_item_cf_recall]
})

# Display the table
print("\nComparison Table:")
print(metrics_df.to_string(index=False))

# Plot the metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# RMSE (lower is better)
axes[0].bar(metrics_df['Method'], metrics_df['RMSE'], color=['royalblue', 'lightgreen', 'salmon'])
axes[0].set_title('RMSE Comparison (lower is better)')
axes[0].set_ylabel('RMSE')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Precision@5 (higher is better)
axes[1].bar(metrics_df['Method'], metrics_df['Precision@5'], color=['royalblue', 'lightgreen', 'salmon'])
axes[1].set_title('Precision@5 Comparison')
axes[1].set_ylabel('Precision@5')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

# Recall@5 (higher is better)
axes[2].bar(metrics_df['Method'], metrics_df['Recall@5'], color=['royalblue', 'lightgreen', 'salmon'])
axes[2].set_title('Recall@5 Comparison')
axes[2].set_ylabel('Recall@5')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Factor Interpretation

Finally, let's interpret the latent factors learned by our matrix factorization model:
1. For each latent factor, identify 3 courses with the highest weight in Q[:, j]
2. Assign possible meanings to the factors
3. Visualize Q as a heatmap (courses × factors)

In [ ]:
# For each latent factor, find the top 3 courses with highest weights
def interpret_factors(Q, course_names, num_factors=None):
    """
    Interpret latent factors by finding top courses for each factor.
    
    Parameters:
    - Q: Course factors matrix
    - course_names: List of course names
    - num_factors: Number of factors to interpret (default: all)
    
    Returns:
    - List of interpretations
    """
    if num_factors is None:
        num_factors = Q.shape[1]
    
    interpretations = []
    
    for factor_idx in range(num_factors):
        # Get the weights for this factor
        factor_weights = Q[:, factor_idx]
        
        # Get indices of courses with highest weights
        top_course_indices = np.argsort(factor_weights)[::-1][:3]
        
        # Get names of top courses
        top_courses = [course_names[idx] for idx in top_course_indices]
        
        # Get weights of top courses
        top_weights = [factor_weights[idx] for idx in top_course_indices]
        
        # Store interpretation
        interpretations.append({
            'factor': factor_idx,
            'top_courses': top_courses,
            'top_weights': top_weights,
            'top_indices': top_course_indices
        })
    
    return interpretations

# Interpret all factors
factor_interpretations = interpret_factors(Q, course_names)

# Print interpretations
print("Latent Factor Interpretations:")
for interp in factor_interpretations:
    factor_idx = interp['factor']
    print(f"\nFactor {factor_idx + 1}:")
    for i, (course, weight) in enumerate(zip(interp['top_courses'], interp['top_weights'])):
        print(f"  {i+1}. {course} (weight: {weight:.4f})")
    
    # Assign a possible meaning to the factor based on top courses
    # This is subjective and would normally involve more analysis
    factor_themes = {
        0: "Programming fundamentals",
        1: "AI and machine learning",
        2: "Software development",
        3: "System design and architecture",
        4: "Applied computer science"
    }
    print(f"  Possible meaning: {factor_themes.get(factor_idx, 'Unknown theme')}")

# Visualize the course-factor matrix (Q) as a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(Q, annot=False, cmap='coolwarm', xticklabels=[f'F{i+1}' for i in range(Q.shape[1])],
            yticklabels=course_names)
plt.title('Course-Factor Matrix Heatmap')
plt.xlabel('Latent Factors')
plt.ylabel('Courses')
plt.tight_layout()
plt.show()

# Create a more focused heatmap showing just the top courses for each factor
# First, create a matrix with just the top courses for each factor
top_courses_per_factor = np.zeros((len(factor_interpretations) * 3, Q.shape[1]))
top_course_names = []

for i, interp in enumerate(factor_interpretations):
    for j, idx in enumerate(interp['top_indices']):
        top_courses_per_factor[i*3 + j, :] = Q[idx, :]
        top_course_names.append(course_names[idx])

plt.figure(figsize=(10, 12))
sns.heatmap(top_courses_per_factor, annot=False, cmap='coolwarm', 
            xticklabels=[f'Factor {i+1}' for i in range(Q.shape[1])],
            yticklabels=top_course_names)
plt.title('Top Courses for Each Factor')
plt.xlabel('Latent Factors')
plt.ylabel('Courses')
plt.tight_layout()
plt.show()

# Visualize the relationship between courses and factors with a network diagram
import networkx as nx

# Create a bipartite graph
G = nx.Graph()

# Add nodes for factors and courses
factor_nodes = [f'Factor {i+1}' for i in range(Q.shape[1])]
course_nodes = course_names

# Add factor nodes
G.add_nodes_from(factor_nodes, bipartite=0)

# Add course nodes
G.add_nodes_from(course_nodes, bipartite=1)

# Add edges between factors and top courses
for interp in factor_interpretations:
    factor = f'Factor {interp["factor"] + 1}'
    for i, idx in enumerate(interp['top_indices']):
        course = course_names[idx]
        weight = interp['top_weights'][i]
        G.add_edge(factor, course, weight=weight)

# Create positions for network visualization (bipartite layout)
pos = {}
factor_pos = np.linspace(0, 1, len(factor_nodes))
course_pos = np.linspace(0, 1, len(course_nodes))

for i, node in enumerate(factor_nodes):
    pos[node] = np.array([0, factor_pos[i]])

for i, node in enumerate(course_nodes):
    pos[node] = np.array([1, course_pos[i]])

# Draw the network
plt.figure(figsize=(10, 12))
nx.draw_networkx_nodes(G, pos, nodelist=factor_nodes, node_color='red', node_size=500, alpha=0.8)
nx.draw_networkx_nodes(G, pos, nodelist=course_nodes, node_color='blue', node_size=300, alpha=0.8)
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=8)
plt.title('Factor-Course Relationships')
plt.axis('off')
plt.tight_layout()
plt.show()